In [ ]:
from glob import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal
from tqdm import tqdm
from simulation import SimulationLongitudinal, LeaderInteraction, LeaderInteractionParameters, \
    IDMPlus, IDMParameters
from load_interactions import load_interaction
from stats import KDE
%matplotlib inline

# Estimate probability density function

In [ ]:
files = glob(os.path.join("data", "8_interactions", "*.hdf5"))

In [ ]:
# Show example
y = load_interaction(files[0])['vehicle_front']
i = y.index[scipy.signal.find_peaks(-y['vel_x'], prominence=1)[0]]
plt.plot(y['vel_x'])
plt.plot(y.loc[i, 'vel_x'], 'r.', ms=20)
plt.xlabel("Time [s]")
plt.ylabel("Speed [m/s]")

In [ ]:
def get_pars(vel_acc):
    if np.max(np.abs(np.diff(vel_acc['vel_x']))) > 1.5:
        return np.zeros((0, 4))
    
    data = vel_acc.copy()
    i = data.index[scipy.signal.find_peaks(-data['vel_x'], prominence=1)[0]]
    data['endspeed'] = np.nan
    data['endtime'] = np.nan
    data.loc[i, 'endspeed'] = data.loc[i, 'vel_x']
    data.loc[i, 'endtime'] = i
    data = data.fillna(method='backfill')
    data = data.dropna()

    data['duration'] = (data['endtime'] - data.index) / 10
    data['reduction'] = data['vel_x'] - data['endspeed']
    data['dmean'] = data['reduction'] / data['duration']
    data = data.drop(i)
    return data[['vel_x', 'acc_x', 'reduction', 'dmean']].values[::10]

In [ ]:
parameters = [get_pars(load_interaction(file)['vehicle_front']) for file in files[:100]]

In [ ]:
kde = KDE(np.concatenate(parameters), scaling=True)
kde.clustering()
kde.compute_bandwidth()

# Simulation

In [ ]:
def leader_parameters(**kwargs):
    return LeaderInteractionParameters(init_position=kwargs["gap"],
                                       init_speed=kwargs["v0_lead"],
                                       init_acceleration=kwargs["a0_lead"],
                                       speed_difference=kwargs["dv"],
                                       duration=kwargs["duration"])

In [ ]:
def follower_parameters(**kwargs):
    return IDMParameters(amin=kwargs["amin"],
                         speed=kwargs["v0_host"],
                         n_reaction=int(kwargs["tr"]*100),
                         init_speed=kwargs["v0_host"],
                         init_position=0)

In [ ]:
s = SimulationLongitudinal(LeaderInteraction(), leader_parameters, IDMPlus(), follower_parameters)

In [ ]:
s.simulation(dict(gap=20, v0_lead=10, a0_lead=0, dv=0, duration=5, v0_host=20, amin=-3, tr=1),
             plot=True)